In [1]:
from tensorflow import keras
import numpy
import pickle
import matplotlib.pyplot as plt

2025-07-04 11:50:32.133092: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 11:50:32.249255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751610032.297397   35660 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751610032.314408   35660 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751610032.420441   35660 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
from keras.preprocessing import sequence
from keras.utils import to_categorical
import random, sys

In [3]:
maxinp=25
maxend=25
rnn_size=512
max_len=maxinp+maxend
batch_norm=False
rnn_layer=3

In [4]:
activation_rnn_size = 40 if maxend else 0

In [5]:
with open("data/vocab-embedding.pkl","rb") as p:
    embedding,id2word,word2dic,glove_id2idx=pickle.load(p)

vocab_size,embedding_size=embedding.shape

In [6]:
print("Vocab Size:",vocab_size)
print("Embedding Size:",embedding_size)

Vocab Size: 40000
Embedding Size: 200


In [7]:
with open("data/vocab-embedding.data.pkl","rb") as p:
    X,Y=pickle.load(p)

nb_unknow_word=10

In [8]:
print('vocabulary size', vocab_size, 'the last %d words can be used as place holders for unknown/oov words' % nb_unknow_word)
print('total number of different words', len(id2word), len(word2dic))
print('number of words outside vocabulary which we can substitute using glove similarity', len(glove_id2idx))
print('number of words that will be regarded as unknown(unk)/out-of-vocabulary(oov)', len(id2word) - vocab_size)

vocabulary size 40000 the last 10 words can be used as place holders for unknown/oov words
total number of different words 5456672 5456672
number of words outside vocabulary which we can substitute using glove similarity 106991
number of words that will be regarded as unknown(unk)/out-of-vocabulary(oov) 5416672


In [9]:
for i in range(nb_unknow_word):
    id2word[vocab_size-1-i] = '<%d>'%i

In [10]:
oov0 = vocab_size-nb_unknow_word

In [11]:
for i in range(oov0, len(id2word)):
    id2word[i] = id2word[i]+'^'

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
len(X_train), len(Y_train), len(X_test), len(Y_test)

(800000, 800000, 200000, 200000)

Deallocating Memory

In [13]:
del X
del Y

In [14]:
empty = 0
eos = 1
id2word[empty] = '_'
id2word[eos] = '~'

In [15]:
def prt(label, x):
    print(label + ':', ' '.join(id2word[w] for w in x))

In [16]:
i = 334
prt('H',Y_train[i])
prt('D',X_train[i])

H: AFL finals 2015: Hawthorn defeat Fremantle,^ stay on track to make it three-in-a-row^
D: HAWTHORN^ 5.2 7.2 11.3^ 15.4^ (94)^ FREMANTLE^ 2.1 4.4 8.4 10.7^ (67)^ Goals:^ Hawthorn:^ C Rioli^ 3, J Roughead^ 2, M Suckling^ 2, R Schoenmakers^ 2, B Hill, B McEvoy,^ D Hale,^ I Smith, L Breust,^ T Duryea.^ Fremantle:^ M Walters 3, J Griffin 2, C Mayne,^ H Ballantyne,^ M Barlow,^ M Pavlich,^ S Hill. BEST:^ Hawthorn:^ S Mitchell, L Hodge,^ T Duryea,^ I Smith, C Rioli,^ B Lake. Fremantle:^ D Mundy,^ L Spurr,^ N Fyfe,^ G Ibbotson,^ M Walters,^ J Griffin.^ Umpires:^ Matt Stevic,^ Jeff Dalgleish,^ Brett Rosebury.^ Official Crowd:^ 41,508^ at Domain Stadium. Hawthorn's^ chances of an amazing premiership three-peat^ remain very much intact. The Hawks withstood^ a dogged Fremantle to win their preliminary final by 27 points at Domain Stadium on Friday night. The Hawks led by 17 points at the final term, but that was scaled back to just nine points very quickly after the restart and the Dockers seemed

## Model

In [17]:
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Dropout,Dense,Activation,LSTM,Embedding,Input
from keras.regularizers import l2

In [18]:
seed=42
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0

In [19]:
if weight_decay:
    regularizer =l2(weight_decay)
else:
    regularizer = None

In [21]:
print(embedding)

[[-0.16595946  0.59620893  0.30688367 ...  0.52530101  0.51204139
   0.37022188]
 [ 0.18788074 -0.55010338 -0.44760057 ... -0.09392734  0.3318545
   0.3367116 ]
 [-0.52499131  0.5325006   0.00694789 ... -0.43345821 -0.40704285
  -0.60734311]
 ...
 [ 0.08179     0.33201     0.44588    ...  0.049709    0.42913
   0.59722   ]
 [ 0.26888187  0.06740113  0.0574175  ...  0.23792921 -0.13171033
  -0.15737504]
 [ 0.30236874  0.30146587  0.44450368 ... -0.35245502 -0.09802368
   0.43994565]]
